In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np
from data_extraction import drought_data

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
input_file = "drought_data/us-droughts.csv"
droughts_df=pd.read_csv(input_file)
droughts_df = droughts_df.drop(droughts_df[['validStart', 'validEnd', 'domStatisticFormatID']], axis=1)
droughts_df.releaseDate = pd.to_datetime(droughts_df.releaseDate, format='%Y-%m-%d')

In [3]:
droughts_df["D0"]=droughts_df["D0"]-droughts_df["D1"]
droughts_df["D1"]=droughts_df["D1"]-droughts_df["D2"]
droughts_df["D2"]=droughts_df["D2"]-droughts_df["D3"]
droughts_df["D3"]=droughts_df["D3"]-droughts_df["D4"]
#adding a drought level based on https://www.kaggle.com/balagpdy/heatmap-animation-us-drought-map
### Calculate drought level (when NONE is 100% => 0, if D4 is 100% => 5, and linearly between 0 and 5)
droughts_df["Level"]=(droughts_df["D0"]+droughts_df["D1"]*2+droughts_df["D2"]*3+droughts_df["D3"]*4+droughts_df["D4"]*5)/100
droughts_df

,releaseDate,FIPS,county,state,NONE,D0,D1,D2,D3,D4,Level
0,2000-11-07,2013,Aleutians East Borough,AK,100.00,0.00,0.0,0.0,0.0,0.0,0.0000
1,2000-10-31,2013,Aleutians East Borough,AK,100.00,0.00,0.0,0.0,0.0,0.0,0.0000
2,2000-10-24,2013,Aleutians East Borough,AK,100.00,0.00,0.0,0.0,0.0,0.0,0.0000
3,2000-10-17,2013,Aleutians East Borough,AK,100.00,0.00,0.0,0.0,0.0,0.0,0.0000
4,2000-10-10,2013,Aleutians East Borough,AK,100.00,0.00,0.0,0.0,0.0,0.0,0.0000
...,...,...,...,...,...,...,...,...,...,...,...
2829496,2015-12-08,56045,Weston County,WY,29.45,70.55,0.0,0.0,0.0,0.0,0.7055
2829497,2015-12-01,56045,Weston County,WY,29.45,70.55,0.0,0.0,0.0,0.0,0.7055
2829498,2015-11-24,56045,Weston County,WY,29.45,70.55,0.0,0.0,0.0,0.0,0.7055
2829499,2015-11-17,56045,Weston County,WY,29.45,70.55,0.0,0.0,0.0,0.0,0.7055


In [4]:
input_file = "drought_data/county_info_2016.csv"
#To read this file properly, due to the format it is recommended that you copy its contents to a new excel file.
#There seems to be a format issue.
county_df=pd.read_csv(input_file)
county_df



,USPS,GEOID,ANSICODE,NAME,ALAND,AWATER,ALAND_SQMI,AWATER_SQMI,INTPTLAT,INTPTLONG
0,AL,1001,161526,Autauga County,1.539609e+09,25749942,594.446,9.942,32.532237,-86.646440
1,AL,1003,161527,Baldwin County,4.117584e+09,1133130502,1589.808,437.504,30.659218,-87.746067
2,AL,1005,161528,Barbour County,2.291819e+09,50864677,884.876,19.639,31.870253,-85.405104
3,AL,1007,161529,Bibb County,1.612482e+09,9287974,622.583,3.586,33.015893,-87.127148
4,AL,1009,161530,Blount County,1.670042e+09,15077458,644.807,5.821,33.977358,-86.566440
...,...,...,...,...,...,...,...,...,...,...
3215,PR,72145,1804553,Vega Baja Municipio,1.187668e+08,57805868,45.856,22.319,18.455027,-66.397773
3216,PR,72147,1804554,Vieques Municipio,1.315278e+08,552206400,50.783,213.208,18.125418,-65.432474
3217,PR,72149,1804555,Villalba Municipio,9.229857e+07,3622637,35.637,1.399,18.130718,-66.472244
3218,PR,72151,1804556,Yabucoa Municipio,1.430052e+08,72592521,55.215,28.028,18.059858,-65.859871


In [5]:
droughtwcount_df=pd.merge(droughts_df,county_df,how='inner',left_on="FIPS",right_on="GEOID",sort=False)

In [6]:
droughtwcount_dfm = droughtwcount_df.set_index('releaseDate').groupby(['FIPS']).resample('M').mean()

In [7]:
#droughtwcount_dfm = droughtwcount_df.set_index('releaseDate').groupby(['FIPS']).resample('M').mean()
droughtwcount_dfm

FIPS     NONE       D0      D1     D2   D3   D4    Level  \
FIPS  releaseDate                                                              
1001  2000-01-31    1001    0.000   25.000  75.000  0.000  0.0  0.0  1.75000   
      2000-02-29    1001    0.000  100.000   0.000  0.000  0.0  0.0  1.00000   
      2000-03-31    1001    0.000   68.145  31.855  0.000  0.0  0.0  1.31855   
      2000-04-30    1001    1.775   98.225   0.000  0.000  0.0  0.0  0.98225   
      2000-05-31    1001    0.000   19.048  78.608  2.344  0.0  0.0  1.83296   
...                  ...      ...      ...     ...    ...  ...  ...      ...   
72153 2016-07-31   72153  100.000    0.000   0.000  0.000  0.0  0.0  0.00000   
      2016-08-31   72153  100.000    0.000   0.000  0.000  0.0  0.0  0.00000   
      2016-09-30   72153  100.000    0.000   0.000  0.000  0.0  0.0  0.00000   
      2016-10-31   72153  100.000    0.000   0.000  0.000  0.0  0.0  0.00000   
      2016-11-30   72153  100.000    0.000   0.000  0.000  0.0  0.0  0.00000   

                   GEOID  ANSICODE         ALAND    AWATER  ALAND_SQMI  \
FIPS  releaseDate                                                        
1001  2000-01-31    1001    161526  1.539609e+09  25749942     594.446   
      2000-02-29    1001    161526  1.539609e+09  25749942     594.446   
      2000-03-31    1001    161526  1.539609e+09  25749942     594.446   
      2000-04-30    1001    161526  1.539609e+09  25749942     594.446   
      2000-05-31    1001    161526  1.539609e+09  25749942     594.446   
...                  ...       ...           ...       ...         ...   
72153 2016-07-31   72153   1804557  1.753719e+08   1625260      67.711   
      2016-08-31   72153   1804557  1.753719e+08   1625260      67.711   
      2016-09-30   72153   1804557  1.753719e+08   1625260      67.711   
      2016-10-31   72153   1804557  1.753719e+08   1625260      67.711   
      2016-11-30   72153   1804557  1.753719e+08   1625260      67.711   

                   AWATER_SQMI   INTPTLAT  \
FIPS  releaseDate                           
1001  2000-01-31         9.942  32.532237   
      2000-02-29         9.942  32.532237   
      2000-03-31         9.942  32.532237   
      2000-04-30         9.942  32.532237   
      2000-05-31         9.942  32.532237   
...                        ...        ...   
72153 2016-07-31         0.628  18.085669   
      2016-08-31         0.628  18.085669   
      2016-09-30         0.628  18.085669   
      2016-10-31         0.628  18.085669   
      2016-11-30         0.628  18.085669   

                   INTPTLONG                                                                                                                 
FIPS  releaseDate                                                                                                                            
1001  2000-01-31                                          -86.646440                                                                         
      2000-02-29                                          -86.646440                                                                         
      2000-03-31                                          -86.646440                                                                         
      2000-04-30                                          -86.646440                                                                         
      2000-05-31                                          -86.646440                                                                         
...                                                              ...                                                                         
72153 2016-07-31                                          -66.857901                                                                         
      2016-08-31                                          -66.857901                                                                         
      2016

In [8]:

#droughtwcount_df.rename(columns={"INTPTLONG                                                                                                               ":"INTPTLONG"})
droughtwcount_dfm2=droughtwcount_dfm[["FIPS","NONE","D0","D1","D2","D3","D4","Level","ALAND_SQMI","AWATER_SQMI","INTPTLAT","INTPTLONG                                                                                                               "]]
droughtwcount_dfm2.rename(columns={"INTPTLONG                                                                                                               ":"INTPTLONG"})


FIPS     NONE       D0      D1     D2   D3   D4    Level  \
FIPS  releaseDate                                                              
1001  2000-01-31    1001    0.000   25.000  75.000  0.000  0.0  0.0  1.75000   
      2000-02-29    1001    0.000  100.000   0.000  0.000  0.0  0.0  1.00000   
      2000-03-31    1001    0.000   68.145  31.855  0.000  0.0  0.0  1.31855   
      2000-04-30    1001    1.775   98.225   0.000  0.000  0.0  0.0  0.98225   
      2000-05-31    1001    0.000   19.048  78.608  2.344  0.0  0.0  1.83296   
...                  ...      ...      ...     ...    ...  ...  ...      ...   
72153 2016-07-31   72153  100.000    0.000   0.000  0.000  0.0  0.0  0.00000   
      2016-08-31   72153  100.000    0.000   0.000  0.000  0.0  0.0  0.00000   
      2016-09-30   72153  100.000    0.000   0.000  0.000  0.0  0.0  0.00000   
      2016-10-31   72153  100.000    0.000   0.000  0.000  0.0  0.0  0.00000   
      2016-11-30   72153  100.000    0.000   0.000  0.000  0.0  0.0  0.00000   

                   ALAND_SQMI  AWATER_SQMI   INTPTLAT  INTPTLONG  
FIPS  releaseDate                                                 
1001  2000-01-31      594.446        9.942  32.532237 -86.646440  
      2000-02-29      594.446        9.942  32.532237 -86.646440  
      2000-03-31      594.446        9.942  32.532237 -86.646440  
      2000-04-30      594.446        9.942  32.532237 -86.646440  
      2000-05-31      594.446        9.942  32.532237 -86.646440  
...                       ...          ...        ...        ...  
72153 2016-07-31       67.711        0.628  18.085669 -66.857901  
      2016-08-31       67.711        0.628  18.085669 -66.857901  
      2016-09-30       67.711        0.628  18.085669 -66.857901  
      2016-10-31       67.711        0.628  18.085669 -66.857901  
      2016-11-30       67.711        0.628  18.085669 -66.857901  

[652239 rows x 12 columns]

In [9]:
droughtwcount_dfm2=droughtwcount_dfm2.dropna()
droughtwcount_dfm2=droughtwcount_dfm2.reset_index(level=1)
droughtwcount_dfm2=droughtwcount_dfm2.drop("FIPS",axis=1)
droughtwcount_dfm2.reset_index(inplace=True)
droughtwcount_dfm2
#dr_m.reset_index(level=1)

,FIPS,releaseDate,NONE,D0,D1,D2,D3,D4,Level,ALAND_SQMI,AWATER_SQMI,INTPTLAT,INTPTLONG
0,1001,2000-01-31,0.000,25.000,75.000,0.000,0.0,0.0,1.75000,594.446,9.942,32.532237,-86.646440
1,1001,2000-02-29,0.000,100.000,0.000,0.000,0.0,0.0,1.00000,594.446,9.942,32.532237,-86.646440
2,1001,2000-03-31,0.000,68.145,31.855,0.000,0.0,0.0,1.31855,594.446,9.942,32.532237,-86.646440
3,1001,2000-04-30,1.775,98.225,0.000,0.000,0.0,0.0,0.98225,594.446,9.942,32.532237,-86.646440
4,1001,2000-05-31,0.000,19.048,78.608,2.344,0.0,0.0,1.83296,594.446,9.942,32.532237,-86.646440
...,...,...,...,...,...,...,...,...,...,...,...,...,...
652234,72153,2016-07-31,100.000,0.000,0.000,0.000,0.0,0.0,0.00000,67.711,0.628,18.085669,-66.857901
652235,72153,2016-08-31,100.000,0.000,0.000,0.000,0.0,0.0,0.00000,67.711,0.628,18.085669,-66.857901
652236,72153,2016-09-30,100.000,0.000,0.000,0.000,0.0,0.0,0.00000,67.711,0.628,18.085669,-66.857901
652237,72153,2016-10-31,100.000,0.000,0.000,0.000,0.0,0.0,0.00000,67.711,0.628,18.085669,-66.857901


In [10]:
test=drought_data()
test


,FIPS,releaseDate,NONE,D0,D1,D2,D3,D4,Level,ALAND_SQMI,AWATER_SQMI,INTPTLAT,INTPTLONG
0,1001,2000-01-31,0.000,25.000,75.000,0.000,0.0,0.0,1.75000,594.446,9.942,32.532237,-86.646440
1,1001,2000-02-29,0.000,100.000,0.000,0.000,0.0,0.0,1.00000,594.446,9.942,32.532237,-86.646440
2,1001,2000-03-31,0.000,68.145,31.855,0.000,0.0,0.0,1.31855,594.446,9.942,32.532237,-86.646440
3,1001,2000-04-30,1.775,98.225,0.000,0.000,0.0,0.0,0.98225,594.446,9.942,32.532237,-86.646440
4,1001,2000-05-31,0.000,19.048,78.608,2.344,0.0,0.0,1.83296,594.446,9.942,32.532237,-86.646440
...,...,...,...,...,...,...,...,...,...,...,...,...,...
652234,72153,2016-07-31,100.000,0.000,0.000,0.000,0.0,0.0,0.00000,67.711,0.628,18.085669,-66.857901
652235,72153,2016-08-31,100.000,0.000,0.000,0.000,0.0,0.0,0.00000,67.711,0.628,18.085669,-66.857901
652236,72153,2016-09-30,100.000,0.000,0.000,0.000,0.0,0.0,0.00000,67.711,0.628,18.085669,-66.857901
652237,72153,2016-10-31,100.000,0.000,0.000,0.000,0.0,0.0,0.00000,67.711,0.628,18.085669,-66.857901


In [14]:
test1 = test.loc[test["Level"] > 4, :]
test1

,FIPS,releaseDate,NONE,D0,D1,D2,D3,D4,Level,ALAND_SQMI,AWATER_SQMI,INTPTLAT,INTPTLONG
6,1001,2000-07-31,0.0,0.0,0.0,25.5125,31.6525,42.8350,4.173225,594.446,9.942,32.532237,-86.64644
7,1001,2000-08-31,0.0,0.0,0.0,0.0000,0.0000,100.0000,5.000000,594.446,9.942,32.532237,-86.64644
8,1001,2000-09-30,0.0,0.0,0.0,0.0000,6.9825,93.0175,4.930175,594.446,9.942,32.532237,-86.64644
9,1001,2000-10-31,0.0,0.0,0.0,0.0000,11.1740,88.8260,4.888260,594.446,9.942,32.532237,-86.64644
91,1001,2007-08-31,0.0,0.0,0.0,0.0000,0.2625,99.7375,4.997375,594.446,9.942,32.532237,-86.64644
...,...,...,...,...,...,...,...,...,...,...,...,...,...
636357,56045,2012-12-31,0.0,0.0,0.0,0.0000,20.3700,79.6300,4.796300,2398.091,1.931,43.846213,-104.57002
636358,56045,2013-01-31,0.0,0.0,0.0,0.0000,20.3700,79.6300,4.796300,2398.091,1.931,43.846213,-104.57002
636359,56045,2013-02-28,0.0,0.0,0.0,0.0000,20.3700,79.6300,4.796300,2398.091,1.931,43.846213,-104.57002
636360,56045,2013-03-31,0.0,0.0,0.0,0.0000,20.3700,79.6300,4.796300,2398.091,1.931,43.846213,-104.57002
